<img src='h1.png'></img>

<img src='h2.png'></img>

In [1]:
import numpy as np
from scipy.special import binom as C

In [148]:
M = 100 #число хромосом
mu = 2 #частота мутаций
K = 10 #количество ответвлений, считая от изначального предка к конечным потомкам = количество конечных потомков

In [149]:
T = np.array([0, .04, .01, .07], dtype=float) #периоды времени
N = np.array([2, 5, 10, 6], dtype=int) #размеры популяции

In [150]:
def v(t):
    '''
    размер популяции в данный момент времени t
    '''
    cond = list()
    for i in range(len(T) - 1):
        cond.append(T[i] <= t < T[i + 1])
    cond.append(t >= T[-1])
    return np.piecewise(t, cond, N)

In [151]:
def w(K, t):
    '''
    интенсивность процесса Пуассона
    K - количество конечных потомков
    t - время
    '''
    return K * mu + 1 / v(t) * C(K, 2)

In [152]:
def simulation(K, mu, T, N):
    '''
    коалесцентная модель
    K - количество конечных потомков
    mu - частота мутаций
    T - периоды времени
    N - размеры популяции
    
    возвращает дерево в виде вложенных списков и времена Пуассоновского процесса для мутации и коалесценции
    '''
    decendents = list([{i:{}}] for i in range(K)) #основа будущего дерева
    np.random.seed(1)
    t = 0
    t_mut = list()
    t_coal = list()
    while K > 1:
        W = w(K, t)
        weight = K * mu
        
        '''
        генерация события мутации, используя распределение Бернулли
        вес weight = K * mu
        вероятность weight / w(K, t)
        
        используется биномиальное распределение с числом испытаний = 1,
        поэтому количество успехов может быть равно либо 1, либо 0
        
        из определения распределения Бернулли неудача события мутации даёт успех события коалесценции 
        '''
        if np.random.binomial(1, weight / W):
            t_mut.append(t)
            #случайно выбираем одного потомка, который оказался подвержен мутации
            l = np.random.randint(0, K)
            #случайно выбираем хромосому, которая оказалась подвержена мутации
            p = np.random.random()
            #всем потомкам особи l передаём эту мутацию - работем внутри ветки decendents[l]
            
            #идём в l список, в нём выбираем 0 элемент - это словарь. 
            #берём ключ l - такой точно есть - изначально номера особей и их порядок в списке совпадают.
            decendents[l][0][l][p] = 1
        else:
            t_coal.append(t)
            l1, l2 = np.random.choice(K, 2, replace=False)
            lmin = min(l1, l2) 
            lmax = max(l1, l2)
            K -= 1
            ancestors = list([{i:{}}] for i in range(K))
            for i in range(lmin):
                ancestors[i].append(decendents[i])
            #чтобы провести даньнейшую связь предков и потомков, проведём замену
            decendents[lmin + 1], decendents[lmax] = decendents[lmax], decendents[lmin + 1]
            ancestors[lmin].append(decendents[lmin])
            ancestors[lmin].append(decendents[lmin + 1])
            for i in range(lmin + 1, K):
                ancestors[i].append(decendents[i + 1])
            decendents = ancestors
        
        '''
        генерация времени до следующего события Пуассоновского процесса
        время имеет экспоненциальное распределение
        параметр равен интенсивности Пуассоновского процесса, т.е. w(K,t)
        '''
        t += np.random.exponential(1 / W)
        
    return decendents, t_mut, t_coal

In [153]:
tree = simulation(K, mu, T, N)[0]

In [154]:
tree

[[{0: {}},
  [{0: {0.6199557183813798: 1}},
   [{0: {0.01864728937294302: 1,
      0.028306488020794607: 1,
      0.2943594751304114: 1}},
    [{0: {}},
     [{0: {}},
      [{0: {}},
       [{0: {0.10233442882782584: 1}}, [{0: {}}, [{0: {}}, [{0: {}}]]]]]]],
    [{2: {0.32664490177209615: 1}},
     [{2: {}},
      [{2: {}},
       [{2: {0.07336417174259957: 1, 0.9888610889064947: 1}},
        [{2: {}}, [{2: {}}, [{2: {}}], [{3: {}}]]]],
       [{3: {0.540600508823644: 1, 0.9168613345297285: 1}},
        [{3: {}}, [{3: {}}, [{4: {}}]], [{7: {}}, [{8: {}}]]]]]]]],
   [{2: {0.0915564416778033: 1,
      0.15213715839256237: 1,
      0.7554630526024664: 1,
      0.9371903244000067: 1}},
    [{3: {0.7715949980443677: 1, 0.9294372337437613: 1}},
     [{4: {}},
      [{4: {0.35653172355805496: 1, 0.6236722070556089: 1}},
       [{5: {0.13747470414623753: 1, 0.5148891120583086: 1}},
        [{5: {}}, [{6: {}}, [{7: {}}]]]]],
      [{5: {}},
       [{6: {0.28777533858634874: 1}}, [{7: {}}, [{8:

### Коалесценция
На каждом поколении своя нумерация. Принадлежность потомка непосредственному предку обозначается их нахождением в одном списке, причём потомок имеет на один уровень вложенности больше.

Например:

[0, [1] ] - предок 0 со своим потомком 1.

[0, [1, [ [1], [2] ] ] ] - предок 0 со своим потомком 1, у которого потомки 1 и 2.

### Добавляем мутацию
Теперь каждая особь это не число, а словарь из словарей. Ключ внешнего словаря - номер особи. Значения внешнего словаря - они же ключи внутренних словарей - хромосомы. Значение внутренного словаря - 0 или 1 - наличие мутации у данной хромосомы.